## 4.Modeling

This stage we apply the model on the dataset prepared on the last stage. First, we will make a simple baseline. The test data set doesn't have a target to check.

A)Baseline

For the baseline we will use the LSTM with dense layers at the end. We also will submit the results on the kaggle with the test set.

In [1]:
from dask.distributed import Client
import dask.dataframe as dd
import logging
import numpy as np
import pandas as pd
import tensorflow as tf

2023-08-24 06:07:25.517831: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 06:07:26.222002: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import pyarrow.parquet as pq
MAX_EPOCHS=10
patience=2
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.8),
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.7),
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.6),
    tf.keras.layers.LSTM(32, return_sequences=False, dropout=0.5),
    #tf.keras.layers.Flatten(),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=128, activation= tf.keras.activations.relu),
    tf.keras.layers.Dense(units=64, activation= tf.keras.activations.relu),
    tf.keras.layers.Dense(units=1, activation= tf.keras.activations.sigmoid)
])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='recall', patience=patience, mode='min')
lstm_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.AdamW(use_ema=True),
              metrics=['acc',tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
parquet_file = pq.ParquetFile('./merged_train/merged_train.parquet')
history = []
for me in range(MAX_EPOCHS):
    for batch in parquet_file.iter_batches(batch_size=650000):
        df = batch.to_pandas('./merged_train/merged_train.parquet')
        #data = np.array(data, dtype=np.float32)
        ds_train = tf.keras.utils.timeseries_dataset_from_array(data=df.iloc[:, 2:-1].to_numpy(),
                                                              targets=df.iloc[:, -1].to_numpy(),
                                                              sequence_length=13,
                                                              sequence_stride=13,
                                                              shuffle=False,
                                                              batch_size=32,)
        history.append( lstm_model.fit(ds_train, epochs=1,shuffle=False,verbose=1,
                            callbacks=[early_stopping]))

2023-08-24 06:07:53.876764: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3114 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 980, pci bus id: 0000:65:00.0, compute capability: 5.2


TypeError: Cannot convert str to pyarrow.lib.MemoryPool

Exception ignored in: 'pyarrow.lib._convert_pandas_options'
Traceback (most recent call last):
  File "/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/pyarrow/pandas_compat.py", line 1168, in _table_to_blocks
    result = pa.lib.table_to_blocks(options, block_table, categories,
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Cannot convert str to pyarrow.lib.MemoryPool
2023-08-24 06:08:05.423740: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-08-24 06:08:05.544519: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f5378173810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-24 06:08:05.544549: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 980, Compute Capability 5.2
2023-08-24 06:08:05.548526: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling 

1563/1563 [==============================] - 19s 8ms/step - loss: 0.3824 - acc: 0.8105 - recall: 0.6153 - precision: 0.6366


TypeError: Cannot convert str to pyarrow.lib.MemoryPool

...

TypeError: Cannot convert str to pyarrow.lib.MemoryPool

Exception ignored in: 'pyarrow.lib._convert_pandas_options'
Traceback (most recent call last):
  File "/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/pyarrow/pandas_compat.py", line 1168, in _table_to_blocks
    result = pa.lib.table_to_blocks(options, block_table, categories,
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Cannot convert str to pyarrow.lib.MemoryPool


1563/1563 [==============================] - 12s 8ms/step - loss: 0.3112 - acc: 0.8548 - recall: 0.7257 - precision: 0.7158


TypeError: Cannot convert str to pyarrow.lib.MemoryPool

Exception ignored in: 'pyarrow.lib._convert_pandas_options'
Traceback (most recent call last):
  File "/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/pyarrow/pandas_compat.py", line 1168, in _table_to_blocks
    result = pa.lib.table_to_blocks(options, block_table, categories,
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Cannot convert str to pyarrow.lib.MemoryPool


In [ ]:
lstm_model.save_weights('lstm_model')

In [ ]:
parquet_file = pq.ParquetFile('./unified_test/test.parquet')
count = 0
for batch in parquet_file.iter_batches(batch_size=130000):
    df = batch.to_pandas()
    customers = df['customer_ID'].copy()
    test = pd.DataFrame(customers.unique().to_numpy().reshape((-1,1)),columns=['customer_ID'])
    
    #data = np.array(data, dtype=np.float32)
    ds_test = tf.keras.utils.timeseries_dataset_from_array(data=df.iloc[:, 2:].to_numpy(),
                                                          targets=None,
                                                          sequence_length=13,
                                                          sequence_stride=13,
                                                          shuffle=False,
                                                          batch_size=1)
    predictions = lstm_model.predict(ds_test)
    test['predictions'] = np.array(predictions).reshape((-1,1))
    test['predictions'] = test['predictions'].apply(lambda x: 1 if x >=0.5 else 0)
    test.to_parquet(f'../data/submit_data/answer{count}.parquet')
    count += 1

In [ ]:
sub = pd.read_parquet("../data/submit_data/")

In [ ]:
sub = sub.rename({'predictions': 'prediction'}, axis=1)

In [ ]:
sub.to_csv("sub_final.csv", index=False)

In [ ]:
!kaggle competitions submit -c amex-default-prediction -f 'sub_final.csv' -m "LSTM+DENSE"

#### Final score:
#### score:0.44
